# Лабораторная работа №3

# Двойственный симплекс метод

### Выполнил: Яковлев Артур, гр. 853501

### Проверила: Костюкова О.И.

#### Вариант 28

Импортируем все необходимые модули

In [2]:
import numpy as np
import scipy.linalg as sla

Исходные данные для тестового примера:

In [18]:
test_dataset = {
    'A': np.array([[2., -3.,  4., 0., 2.,  0., -2., 7.],
                   [0.,  2., -2., 2., 1., -1.,  4., 3.],
                   [0.,  4.,  1., 1., 3.,  5.,  1., 2.]]),
    'b': np.array([0., -4., -18.]),
    'c': np.array([-2., 15., -11., 8., 3., -2., 13., 7.]),
    'y': np.array([-1., 4., 1.]),
    'jb': [0, 1, 2],
}

Реализуем итерацию нашего алгоритма. Данная функция принимает в себя все исходные данные, обратную матрицу $B$ и вектор $\delta$. Итерация состоит из 9 шагов, каждый из которых приведен в коде ниже.

In [19]:
def iteration(dataset, B, delta):  # B is a np.array, delta is Dict[int, float]
    #Step 1
    x_b = B.dot(dataset['b'])
    print(f'Current plan: {x_b}')
    #Step 2
    if np.all(x_b > -1e-9):
        return None, x_b
    #Step 3
    k = np.argmin(x_b)
    print(f'k: {k}')
    jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['jb']]
    print(f'jn: {jn}')
    mu = {j: B[k, :].dot(dataset['A'][:, j]) for j in jn}
    print(f'mu: {mu}')
    #Step 4
    sigma = {j: -delta[j] / mu[j] if mu[j] < 1e-9 else np.inf for j in jn}
    print(f'sigma: {sigma}')
    #Step 5, 6
    sigma0 = min(sigma, key=sigma.get)
    if sigma[sigma0] == np.inf:
        return None, None
    #Step 7
    idx = dataset['jb'][k]
    del dataset['jb'][k]
    dataset['jb'].append(sigma0)
    dataset['jb'].sort()
    print(f'New jb: {dataset["jb"]}')
    #Step 8
    delta.pop(sigma0)
    for key in delta:
        delta[key] += sigma[sigma0] * mu[key]
    delta[idx] = sigma[sigma0]
    print(f'New delta: {delta}')
    #Step 9
    B = sla.inv(dataset['A'][:, dataset['jb']])
    return B, delta

Далее реализуем сам алгоритм решения. Функция принимает в себя набор исходных данных, состоящих из матрицы $A$, векторов $b$, $c$, $y$ и массива индексов $J_b$, а также максимальное число итераций алгоритма (по умолчанию 20). В основном цикле программы с помощью функции итерации обновляются матрица $B$ и вектор $\delta$ и осуществляется вывод оптимального решения в случае, если оно было получено.

In [20]:
def solve(dataset, max_iter=20):
    B = sla.inv(dataset['A'][:, dataset['jb']])
    jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['jb']]
    init_delta = np.dot(dataset['c'][dataset['jb']], B @ dataset['A'][:, jn]) - dataset['c'][jn]
    delta = {jn[i]: init_delta[i] for i in range(len(jn))}
    for i in range(max_iter):
        print('\n==============================================\n')
        print(f'Iteration {i + 1}\n')
        B, delta = iteration(dataset, B, delta)
        if B is None:
            if delta is None:
                print('\nNo solution\n')
            else:
                plan = np.zeros(dataset['c'].shape)
                plan[dataset['jb']] = delta
                print(f'\nOptimized plan: {plan}')
                print(f"cx: {dataset['c'][dataset['jb']].dot(delta)}")
            return
    print(f'\nBest found plan: {delta}')

Проверим наш алгоритм на тестовом наборе данных

In [21]:
solve(test_dataset)



Iteration 1

Current plan: [-2. -4. -2.]
k: 1
jn: [3, 4, 5, 6, 7]
mu: {3: 0.4, 4: 0.7000000000000001, 5: 0.9, 6: 0.6000000000000001, 7: 0.7000000000000001}
sigma: {3: inf, 4: inf, 5: inf, 6: inf, 7: inf}

No solution



В тестовом примере получили, что решения нет, т. к. все значения в матрице $\sigma$ уходят в бесконечность

Далее рассмотрим задачу для варианта 28. Исходные данные:

In [22]:
var28 = {
    'A': np.array([[ 1.,  2., -1.,  0.,  3.,  2.,  3.,  1.,  0.],
                   [ 1.,  0.,  0.,  1., -1.,  3.,  0.,  7.,  5.],
                   [ 0.,  1., -4.,  1., -3.,  1.,  0.,  1.,  0.],
                   [ 1.,  2., -3.,  1., -1.,  2.,  5.,  0.,  4.]]),
    'b': np.array([2., 3., -3., 3.]),
    'c': np.array([8., 8., 2., 0., 21., 12., 16., 16., 9.]),
    'y': np.array([5., 2., -3., 1.]),
    'jb': [1, 4, 5, 8],  # изменил несколько индексов, чтобы алгоритм не прекращал работу на первой итерации
}

In [23]:
solve(var28)



Iteration 1

Current plan: [ 0.26851852  0.88888889 -0.60185185  1.13888889]
k: 2
jn: [0, 2, 3, 6, 7]
mu: {0: 0.18518518518518517, 2: -0.20370370370370328, 3: 0.2129629629629629, 6: -1.2777777777777777, 7: 2.898148148148148}
sigma: {0: inf, 2: 13.181818181818217, 3: inf, 6: 1.0869565217391308, 7: inf}
New jb: [1, 4, 6, 8]
New delta: {0: -1.0579710144927539, 2: 2.463768115942031, 3: -0.6666666666666664, 7: -1.5072463768115907, 5: 1.0869565217391308}


Iteration 2

Current plan: [-0.80434783  0.73188406  0.47101449  0.74637681]
k: 0
jn: [0, 2, 3, 5, 7]
mu: {0: 0.47826086956521746, 2: -1.826086956521739, 3: 0.49999999999999994, 5: 1.7826086956521743, 7: 2.9347826086956523}
sigma: {0: inf, 2: 1.3492063492063504, 3: inf, 5: inf, 7: inf}
New jb: [2, 4, 6, 8]
New delta: {0: -0.41269841269841223, 3: 0.007936507936508685, 7: 2.4523809523809597, 5: 3.492063492063495, 1: 1.3492063492063504}


Iteration 3

Current plan: [0.44047619 0.41269841 0.40079365 0.68253968]

Optimized plan: [0.         0

Аналогично решим задачу для варианта 29.

In [162]:
var29 = {
    'A': np.array([
         [-2, -1,  1,  -7,  1,  0,  0,  2],
         [-4,  2,  1,   0,  5,  1, -1,  5],
         [ 1,  1,  0,  -1,  0,  3, -1,  1]]),
    'b': np.array([-2, 4, -2]),
    'c': np.array([-12, 2, 2, -6, 10, -1, -9, 8]),
    'y': np.array([1, 2, -1]),
    'jb': [1, 3, 5]
}

In [163]:
solve(var29)



Iteration 1

Current plan: [ 2.82352941 -0.11764706 -1.64705882]
k: 2
jn: [0, 2, 4, 6, 7]
mu: {0: 1.4705882352941178, 2: -0.29411764705882354, 4: -1.2352941176470589, 6: -0.1764705882352941, 7: -0.8823529411764707}
sigma: {0: inf, 2: 3.4, 4: 0.8095238095238095, 6: 45.333333333333336, 7: 3.3999999999999995}
New jb: [1, 3, 4]
New delta: {0: 2.1904761904761907, 2: 0.7619047619047619, 6: 7.857142857142857, 7: 2.2857142857142856, 5: 0.8095238095238095}


Iteration 2

Current plan: [-1.33333333  0.66666667  1.33333333]
k: 0
jn: [0, 2, 5, 6, 7]
mu: {0: 0.9761904761904759, 2: -0.09523809523809519, 5: 2.5238095238095237, 6: -0.8571428571428571, 7: 0.7142857142857144}
sigma: {0: inf, 2: 8.000000000000004, 5: inf, 6: 9.166666666666666, 7: inf}
New jb: [2, 3, 4]
New delta: {0: 10.000000000000002, 6: 0.9999999999999973, 7: 8.000000000000004, 5: 21.00000000000001, 1: 8.000000000000004}


Iteration 3

Current plan: [14.  2. -2.]
k: 2
jn: [0, 1, 5, 6, 7]
mu: {0: 1.25, 1: 2.5, 5: 5.5, 6: -2.0, 7: 2.5

In [17]:
import numpy as np


def double_simplex(A, b, c, y, Jb, log=None):
    m, n = A.shape
    Jb = list(map(lambda q: q - 1, Jb))
    eps = 0.0000001

    iter = 0
    while True:
        iter += 1
        # step 0
        Ab = A[:, Jb]
        B = sla.inv(Ab)
        if B is None:
            return
        coplan = y.dot(A) - c

        if log is not None:
            print('Итерация', iter)
            print('Jb =', Jb)
            print('Ab:', Ab)
            print('B:', B)

        # step 1
        kappa_b = B.dot(b)

        if log is not None:
            print('Базисные компоненты псевдоплана:', kappa_b)

        # step 2
        if all(kp > -eps for kp in kappa_b):

            if log is not None:
                print('Поскольку все компоненты >= 0,')

            answer = [0 for i in range(n)]
            for i, j in zip(Jb, kappa_b):
                answer[i] = j
            print('Ответ:', answer)
            print('c * x:', c.dot(answer))
            return
        else:
            if log is not None:
                print('Не все компоненты >= 0')

        # step 3
        temp = None
        for kp in kappa_b:
            if kp < -eps:
                temp = kp
                break
        k = list(kappa_b).index(temp)

        if log is not None:
            print('k =', k)

        mu = []
        for j in range(n):
            mu.append(B[k].dot(A[:, j]))

        if log is not None:
            print('Находим числа мю:', mu)

        # step 4
        sigmas = []
        Jn = [j for j in range(n) if j not in Jb]
        for j in Jn:
            sigmas.append(-coplan[j] / mu[j] if mu[j] < -eps else float("inf"))
        sigma0 = min(sigmas)

        if log is not None:
            print('Вычисляем шаги:', sigmas)

        # step 5
        if sigma0 == float('inf'):
            print('Поскольку все сигма == Inf, то ограничения прямой задачи несовместны')
            return

        # step 6
        j0 = Jn[sigmas.index(sigma0)]

        if log is not None:
            print('j0 =', j0)

        # step 7
        y = y + sigma0 * B[k]
        Jb[k] = j0

        if log is not None:
            print('Новый двойственный план:', y)
            print('Новый базис', Jb)
            print('Новый коплан', y.dot(A) - c)
            print('---------------------------')


# def double_simplexz(A, b, c, y, Jb):
#     m, n = A.shape
#     Jb = list(map(lambda q: q - 1, Jb))
#     Jn = [j for j in range(n) if j not in Jb]
#     eps = 0.00000001
#
#     Ab = A[:, Jb]
#     B = invert_matrix(Ab)
#     if B is None:
#         return
#     coplan_n = y.dot(A[:, Jn]) - np.array([c[j] for j in Jn])
#
#     while True:
#         # step 1
#         kappa_b = B.dot(b)
#
#         # step 2
#         if all(kp > -eps for kp in kappa_b):
#             answer = [0 for i in range(n)]
#             for i, j in zip(Jb, kappa_b):
#                 answer[i] = j
#             print(answer)
#             print(c.dot(np.array(answer)))
#             return
#
#         # step 3
#         temp = None
#         for kp in kappa_b:
#             if kp < eps:
#                 temp = kp
#                 break
#         k = list(kappa_b).index(temp)
#
#         mu = []
#         for j in range(n):
#             mu.append(B[k].dot(A[:, j]))
#
#         # step 4
#         sigmas_n = []
#         for j in Jn:
#             i = Jn.index(j)
#             sigmas_n.append(-coplan_n[i] / mu[j] if mu[j] < -eps else float("inf"))
#         sigma0 = min(sigmas_n)
#
#         # step 5
#         if sigma0 == float('inf'):
#             print('Ограничения прямой задачи несовместны')
#             return
#
#         # step 6
#         j0 = Jn[sigmas_n.index(sigma0)]
#
#         # step 7
#         Jb[k] = j0
#
#         # step 8
#         for j in Jn:
#             i = Jn.index(j)
#             coplan_n[i] = coplan_n[i] + sigma0 * mu[i]
#         coplan_n[k] = sigma0
#
#         # step 9
#         B = invert_matrix(A[:, Jb])


# вариант 23
# A = np.array([[1, -1, 0, 2, -1, -1, 2, 1, 7],
#      [1, -2, -1, 1, 0, -1, 1, 1, -3],
#      [1, 1, 1, 1, 1, 1, 3, 1, 4],
#      [1, -3, 0, -8, 2, 3, 0, 1, 1]])
# b = np.array([8.00, 5.00, 14.00, 11.00])
# c = np.array([3, 3, 0, 32, -9, -16, 8, 3, 26])
# y = np.array([4, 1, 1, -3])
# Jb = np.array([1, 3, 5, 8])

# пример
# A = np.array([
#      [-2, -1, 1, -7, 0, 0, 0, 2],
#      [4, 2, 1, 0, 1, 5, -1, -5],
#      [1, 1, 0, -1, 0, 3, -1, 1]])
# b = np.array([-2, 4, 3])
# c = np.array([2, 2, 1, -10, 1, 4, -2, -3])
# y = np.array([1, 1, 1])
# Jb = np.array([2, 5, 7])

# задача 1
# A = np.array([
#      [-2, -1, 1, -7, 0, 0, 0, 2],
#      [4, 2, 1, 0, 1, 5, -1, -5],
#      [1, 1, 0, -1, 0, 3, -1, 1]])
# b = np.array([-2, -4, -2])
# c = np.array([5, 2, 3, -16, 1, 3, -3, -12])
# y = np.array([1, 2, -1])
# Jb = np.array([1, 2, 3])

# задача 5
# A = np.array([
#      [3, -1, 10, -7, 1, 0, 0, 2],
#      [7, -2, 14, 8, 0, 12, -11, 0],
#      [1, 1, 0, 1, -4, 3, -1, 1]])
# b = np.array([2, 5, -2])
# c = np.array([36, -12, 66, 76, -5, 77, -76, -7])
# y = np.array([-3, 7, -1])
# Jb = np.array([7, 8, 4])

# задача 4
# A = np.array([
#      [-2, -1, 10, -7, 1, 0, 0, 2],
#      [-4, 2, 3, 0, 5, 1, -1, 0],
#      [1, 1, 0, 1, -4, 3, -1, 1]])
# b = np.array([-2, -5, 2])
# c = np.array([10, -2, -38, 16, -9, -9, -5, -7])
# y = np.array([-3, -2, -1])
# Jb = np.array([2, 8, 5])

# задача 2
# A = np.array([
#      [-2, -1, 1, -7, 1, 0, 0, 2],
#      [-4, 2, 1, 0, 5, 1, -1, 5],
#      [1, 1, 0, -1, 0, 3, -1, 1]])
# b = np.array([-2, 4, -2])
# c = np.array([-12, 2, 2, -6, 10, -1, -9, 8])
# y = np.array([1, 2, -1])
# Jb = np.array([2, 4, 6])

# для всех
A = np.array([
     [2, -3, 4, 0, 2, 0, -2, 7],
     [0, 2, -2, 2, 1, -1, 4, 3],
     [0, 4, 1, 1, 3, 5, 1, 2]])
b = np.array([0, -4, -18])
c = np.array([-2, 15, -11, 8, 3, -2, 13, 7])
y = np.array([-1, 4, 1])
Jb = np.array([1, 2, 3])

# вар 29
# A = np.array([
#     [1,  2, - 1,  0,   3,  2,    3,  1,   5],
#     [1,  0, 0,  -1,  -1,  -3,   0,  -6,   1],
#     [0,  1,  -4,   1,   3,  1,    0,  1,   0],
#     [1,  2,  -3,  1,  -1,  2,    5,  0,   4]
#              ])
# b = np.array([22, 2, 8, 12])
# c = np.array([-9, -13, 19, -5, 2, -7, -32, 11, -28])
# y = np.array([-1, -3, -1, -5])
# Jb = np.array([1, 2, 5, 9])

double_simplex(A, b, c, y, Jb, 'log')

Итерация 1
Jb = [0, 1, 2]
Ab: [[ 2 -3  4]
 [ 0  2 -2]
 [ 0  4  1]]
B: [[ 0.5   0.95 -0.1 ]
 [ 0.    0.1   0.2 ]
 [ 0.   -0.4   0.2 ]]
Базисные компоненты псевдоплана: [-2. -4. -2.]
Не все компоненты >= 0
k = 0
Находим числа мю: [1.0, 0.0, -1.6653345369377348e-16, 1.8, 1.6500000000000001, -1.4500000000000002, 2.7, 6.15]
Вычисляем шаги: [inf, inf, 2.068965517241379, inf, inf]
j0 = 5
Новый двойственный план: [0.03448276 5.96551724 0.79310345]
Новый базис [5, 1, 2]
Новый коплан [ 2.06896552  0.          0.          4.72413793  5.4137931   0.
 11.5862069  12.72413793]
---------------------------
Итерация 2
Jb = [5, 1, 2]
Ab: [[ 0 -3  4]
 [-1  2 -2]
 [ 5  4  1]]
B: [[-0.34482759 -0.65517241  0.06896552]
 [ 0.31034483  0.68965517  0.13793103]
 [ 0.48275862  0.51724138  0.10344828]]
Базисные компоненты псевдоплана: [ 1.37931034 -5.24137931 -3.93103448]
Не все компоненты >= 0
k = 1
Находим числа мю: [0.6206896551724138, 1.0, -2.7755575615628914e-17, 1.517241379310345, 1.7241379310344829, 8.3266